In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

%matplotlib widget

# Creating the signal and kernel

In [3]:
num_samples = 3000
x = np.random.normal(0, 1, size=num_samples)
x.shape

(3000,)

In [4]:
plt.close('all')
plt.title('Random Noise ($\mu=0$, $\sigma=1$)')
plt.plot(x)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
kernel = signal.gaussian(100, 10)
kernel.shape

(100,)

In [6]:
plt.close('all')
plt.title('Gaussian Kernel ($\sigma=10$)')
plt.plot(kernel)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Traditional filtering (Convolution)

In [7]:
M = kernel.shape[0]
x_filtered = np.convolve(x, kernel, mode='full')
x_filtered.shape

(3099,)

In [8]:
plt.close('all')
plt.title('Filtered signal')
plt.plot(x_filtered)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# DFT filtering

In [9]:
# Zero padding
x_padded = np.hstack([x, np.zeros(M-1)])
print(x_padded.shape)
kernel_padded = np.hstack([kernel, np.zeros(x.shape[0]-1)])
print(kernel_padded.shape)

(3099,)
(3099,)


In [10]:
plt.close('all')
plt.figure(figsize=(9,3))
plt.subplot(121)
plt.title('Padded Random Noise ($\mu=0$, $\sigma=1$)')
plt.plot(x_padded)
plt.subplot(122)
plt.title('Padded Gaussian Kernel ($\sigma=10$)')
plt.plot(kernel_padded)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# Filtering by multiplying in the Fourier domain
x_FT = np.fft.fft(x_padded)
print(x_FT.shape)
kernel_FT = np.fft.fft(kernel_padded)
print(kernel_FT.shape)
x_filtered_FT = np.fft.ifft(x_FT * kernel_FT).real
print(x_filtered_FT.shape)

(3099,)
(3099,)
(3099,)


In [19]:
plt.close('all')
plt.figure(figsize=(9,4))
plt.subplot(211)
plt.title('Traditional filtering')
plt.plot(x_filtered)
plt.subplot(212)
plt.title('DFT filtering')
plt.plot(x_filtered_FT)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# DFT filtering by blocks

In [20]:
# Block separation
x_1 = x[:x.shape[0]//3]
x_2 = x[x.shape[0]//3:2*x.shape[0]//3]
x_3 = x[2*x.shape[0]//3:]
print(x_1.shape, x_2.shape, x_3.shape)

(1000,) (1000,) (1000,)


## Method 1: Overlap add
<img src="images/slide_11-5.png" alt="slide 11-5" width="800">

In [21]:
# Zero padding
kernel_padded = np.hstack([kernel, np.zeros(x_1.shape[0]-1)])
print(kernel_padded.shape)
x_1_padded = np.hstack([x_1, np.zeros(M-1)])
print(x_1_padded.shape)
x_2_padded = np.hstack([x_2, np.zeros(M-1)])
print(x_2_padded.shape)
x_3_padded = np.hstack([x_3, np.zeros(M-1)])
print(x_3_padded.shape)

# DFT filtering
kernel_FT = np.fft.fft(kernel_padded)
x_1_FT = np.fft.fft(x_1_padded)
x_1_filtered_FT = np.fft.ifft(x_1_FT * kernel_FT).real
x_2_FT = np.fft.fft(x_2_padded)
x_2_filtered_FT = np.fft.ifft(x_2_FT * kernel_FT).real
x_3_FT = np.fft.fft(x_3_padded)
x_3_filtered_FT = np.fft.ifft(x_3_FT * kernel_FT).real

(1099,)
(1099,)
(1099,)
(1099,)


In [22]:
plt.close('all')
fig, axs = plt.subplots(1, 3, figsize=(9,3))
axs[0].plot(x_1_filtered_FT)
axs[0].set_title('$x_1$')
axs[1].plot(x_2_filtered_FT)
axs[1].set_title('$x_2$')
axs[2].plot(x_3_filtered_FT)
axs[2].set_title('$x_3$')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
x_1_filtered_FT_padded = np.zeros(3*x_1.shape[0]+M-1)
x_2_filtered_FT_padded = np.zeros(3*x_1.shape[0]+M-1)
x_3_filtered_FT_padded = np.zeros(3*x_1.shape[0]+M-1)

# Index shifting
x_1_filtered_FT_padded[:x_1_filtered_FT.shape[0]] = x_1_filtered_FT
x_2_filtered_FT_padded[x_1_filtered_FT.shape[0]-(M-1):2*x_1_filtered_FT.shape[0]-(M-1)] = x_2_filtered_FT
x_3_filtered_FT_padded[2*x_1_filtered_FT.shape[0]-2*(M-1):] = x_3_filtered_FT

# Creation of the final signal by addition of the index-shifted blocks
x_filtered_blocks = x_1_filtered_FT_padded + x_2_filtered_FT_padded + x_3_filtered_FT_padded

In [24]:
indices = np.linspace(0, x_1_filtered_FT_padded.shape[0]-1, x_1_filtered_FT_padded.shape[0])
plt.close('all')
fig, axs = plt.subplots(3, 1, figsize=(9,6))

# Only display the non-zero components of the index-shifted blocks
axs[0].plot(indices[x_1_filtered_FT_padded != 0], x_1_filtered_FT_padded[x_1_filtered_FT_padded != 0], label='$x_1$')
axs[0].plot(indices[x_2_filtered_FT_padded != 0], x_2_filtered_FT_padded[x_2_filtered_FT_padded != 0], label='$x_2$')
axs[0].plot(indices[x_3_filtered_FT_padded != 0], x_3_filtered_FT_padded[x_3_filtered_FT_padded != 0], label='$x_3$')
axs[0].set_title('DFT filtered blocks')
axs[0].legend(loc='upper right')

axs[1].plot(x_filtered_blocks)
axs[1].set_title('DFT filtered signal')

axs[2].plot(x_filtered)
axs[2].set_title('Traditionally filtered signal')

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Method 2: Overlap save
<img src="images/slide_11-6.png" alt="slide 11-6" width="900">

In [25]:
# Creation of extended blocks
kernel_padded = np.hstack([kernel, np.zeros(x_1.shape[0]-1)])
print(kernel_padded.shape)
x_1_extended = np.hstack([np.zeros(M-1), x_1])
print(x_1_extended.shape)
x_2_extended = np.hstack([x_1[-(M-1):], x_2])
print(x_2_extended.shape)
x_3_extended = np.hstack([x_2[-(M-1):], x_3])
print(x_3_extended.shape)
# A 4th block is necessary to get the last M-1-1 values (because of 'full' convolution)
x_4_extended = np.hstack([x_3[-(M-1):], np.zeros(x_3.shape[0])])
print(x_4_extended.shape)

# DFT filtering
x_1_FT = np.fft.fft(x_1_extended)
x_1_filtered_FT = np.fft.ifft(x_1_FT * kernel_FT).real
x_2_FT = np.fft.fft(x_2_extended)
x_2_filtered_FT = np.fft.ifft(x_2_FT * kernel_FT).real
x_3_FT = np.fft.fft(x_3_extended)
x_3_filtered_FT = np.fft.ifft(x_3_FT * kernel_FT).real
x_4_FT = np.fft.fft(x_4_extended)
x_4_filtered_FT = np.fft.ifft(x_4_FT * kernel_FT).real

(1099,)
(1099,)
(1099,)
(1099,)
(1099,)


In [26]:
plt.close('all')
fig, axs = plt.subplots(2, 2, figsize=(9,5))

axs[0][0].plot(x_1_filtered_FT)
axs[0][0].set_title('$x_1$')

axs[0][1].plot(x_2_filtered_FT)
axs[0][1].set_title('$x_2$')

axs[1][0].plot(x_3_filtered_FT)
axs[1][0].set_title('$x_3$')

axs[1][1].plot(x_4_filtered_FT)
axs[1][1].set_title('$x_4$')

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
# Combining the blocks to form the final filtered signal (remove the first M-1 values from all blocks).
x_filtered_blocks = np.hstack([x_1_filtered_FT[M-1:], x_2_filtered_FT[M-1:], x_3_filtered_FT[M-1:], x_4_filtered_FT[M-1:2*(M-1)]])
x_filtered_blocks.shape

(3099,)

In [28]:
indices = np.linspace(-(M-1), x_filtered_blocks.shape[0]-1, x_filtered_blocks.shape[0]+(M-1))
plt.close('all')
fig, axs = plt.subplots(3, 1, figsize=(9,6))

# Discard the first M-1 values since we don't need them
axs[0].plot(indices[M-1:x_1_filtered_FT.shape[0]], x_1_filtered_FT[M-1:], label='$x_1$')
axs[0].plot(indices[x_1_filtered_FT.shape[0]-(M-1):2*x_1_filtered_FT.shape[0]-(M-1)], x_2_filtered_FT, label='$x_2$')
axs[0].plot(indices[2*(x_1_filtered_FT.shape[0]-(M-1)):3*x_1_filtered_FT.shape[0]-2*(M-1)], x_3_filtered_FT, label='$x_3$')
# Only keep the first 2*(M-1) values from the last block since we don't need the rest
axs[0].plot(indices[3*(x_1_filtered_FT.shape[0]-(M-1)):], x_4_filtered_FT[:2*(M-1)], label='$x_4$')
axs[0].set_title('DFT filtered blocks')
axs[0].legend(loc='upper right')

axs[1].plot(x_filtered_blocks)
axs[1].set_title('DFT filtered signal')

axs[2].plot(x_filtered)
axs[2].set_title('Traditionally filtered signal')

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …